# PySpark Demo Plotly Notebook
## Demo
1. Setup Spark
2. Load Kaggle Data
3. Transform Data with Spark SQL
4. Graph Data with Plotly


### Requirements
1. Create a free [Plotly Chart Studio](https://chart-studio.plot.ly) account
2. Generate a Plotly API Key
3. Export Username and API Key as enlivenment variables

_Prepared by: [Gary A. Stafford](https://twitter.com/GaryStafford)   
Associated article: https://wp.me/p1RD28-61V_

### Setup Spark
Setup Spark SparkSession

In [1]:
from pyspark.sql import SparkSession

In [2]:
# reference: https://spark.apache.org/docs/latest/configuration.html#viewing-spark-properties
spark = SparkSession \
    .builder \
    .appName('05_notebook') \
    .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', '05_notebook'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1575409000556'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '44197'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'b814d9d5a2ae'),
 ('spark.ui.showConsoleProgress', 'true')]

### Load Kaggle Dataset
Load the Kaggle dataset from the CSV file, containing ~21K rows, into Spark DataFrame

In [4]:
df1 = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('delimiter', ',') \
    .option('inferSchema', True) \
    .load('BreadBasket_DMS.csv')

print('DataFrame rows: %d' % df1.count())
print('DataFrame schema: %s' % df1)
df1.show(10, False)

DataFrame rows: 21293
DataFrame schema: DataFrame[Date: timestamp, Time: string, Transaction: int, Item: string]
+-------------------+--------+-----------+-------------+
|Date               |Time    |Transaction|Item         |
+-------------------+--------+-----------+-------------+
|2016-10-30 00:00:00|09:58:11|1          |Bread        |
|2016-10-30 00:00:00|10:05:34|2          |Scandinavian |
|2016-10-30 00:00:00|10:05:34|2          |Scandinavian |
|2016-10-30 00:00:00|10:07:57|3          |Hot chocolate|
|2016-10-30 00:00:00|10:07:57|3          |Jam          |
|2016-10-30 00:00:00|10:07:57|3          |Cookies      |
|2016-10-30 00:00:00|10:08:41|4          |Muffin       |
|2016-10-30 00:00:00|10:13:03|5          |Coffee       |
|2016-10-30 00:00:00|10:13:03|5          |Pastry       |
|2016-10-30 00:00:00|10:13:03|5          |Bread        |
+-------------------+--------+-----------+-------------+
only showing top 10 rows



### Transform Data with Spark SQL
Transform the DataFrame's bakery data using Spark SQL

In [5]:
df1.createOrReplaceTempView('tmp_bakery')
df2 = spark.sql("SELECT date, transaction, item " +
                "FROM tmp_bakery " +
                "WHERE item NOT LIKE 'NONE' " +
                "ORDER BY transaction")

print('DataFrame rows: %d' % df2.count())
df2.show(5, False)

DataFrame rows: 20507
+-------------------+-----------+-------------+
|date               |transaction|item         |
+-------------------+-----------+-------------+
|2016-10-30 00:00:00|1          |Bread        |
|2016-10-30 00:00:00|2          |Scandinavian |
|2016-10-30 00:00:00|2          |Scandinavian |
|2016-10-30 00:00:00|3          |Hot chocolate|
|2016-10-30 00:00:00|3          |Jam          |
+-------------------+-----------+-------------+
only showing top 5 rows



In [6]:
df2.createOrReplaceTempView('tmp_bakery')

df3 = spark.sql("SELECT date, count(*) as count " +
                "FROM tmp_bakery " +
                "WHERE date >= '2017-01-01' " +
                "GROUP BY date " +
                "ORDER BY date")

print('DataFrame rows: %d' % df3.count())
df3.show(5, False)

DataFrame rows: 98
+-------------------+-----+
|date               |count|
+-------------------+-----+
|2017-01-01 00:00:00|1    |
|2017-01-03 00:00:00|87   |
|2017-01-04 00:00:00|76   |
|2017-01-05 00:00:00|95   |
|2017-01-06 00:00:00|84   |
+-------------------+-----+
only showing top 5 rows



### Graph Data with Plotly
Use [Plotly](https://plot.ly/python/) to create a chart showing bakery items sold over time  
Demostrates linear fit and data smoothing:
* [Plotly Python Open Source Graphing Library](https://plot.ly/python/)
* [Smoothing in Python](https://plot.ly/python/smoothing/)
* [Linear Fit in Python](https://plot.ly/python/linear-fits/)

In [7]:
import os
from dotenv import load_dotenv
import chart_studio.tools
import chart_studio.plotly as py
import plotly.graph_objs as go
from numpy import arange
from scipy import stats, signal
import warnings

warnings.filterwarnings('ignore')

In [8]:
# load plotly credentials
load_dotenv()
chart_studio.tools.set_credentials_file(
    username=os.getenv('PLOTLY_USERNAME'), 
    api_key=os.getenv('PLOTLY_API_KEY'))

In [9]:
pdf4 = df3.toPandas()

# Generated linear fit
xi = arange(0, len(pdf4.index))
slope, intercept, r_value, p_value, std_err = stats.linregress(xi, pdf4['count'])
line = slope * xi + intercept

layout = dict(title='2017 Bakery Sales',
              xaxis=dict(
                  title='Month',
                  showgrid=True,
                  zeroline=True,
                  showline=True,
                  ticks='outside',
                  tickangle=45,
                  showticklabels=True),
              yaxis=dict(
                  title='Items Sold/Day',
                  showgrid=True,
                  zeroline=True,
                  showline=True,
                  ticks='outside',
                  showticklabels=True))

trace1 = go.Bar(x=pdf4['date'], y=pdf4['count'], name='Items Sold')
trace2 = go.Scatter(x=pdf4['date'], y=line, mode='lines', name='Linear Fit')
trace3 = go.Scatter(x=pdf4['date'], y=signal.savgol_filter(pdf4['count'], 53, 3),
                    mode='lines', name='Savitzky-Golay')
data = [trace1, trace2, trace3]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar.html')